# Proyecto de Adquisición de Datos Parte I- Análisis Computacional de datos 

## Integrantes del Grupo
- **Carlos Renato Guerra Marquina**
- **Adrian Aron Urbina Mendoza**
- **Renzo Chen Heng Liang Corrales**

---

# INTRODUCCIÓN Y MARCO TEÓRICO

## Descripción

Hemos desarrollado un sistema automatizado de extracción de datos de productos de celulares desde MercadoLibre Perú. Nuestro objetivo principal es recopilar información estructurada para análisis de mercado mediante técnicas de web scraping.

## Justificación

Seleccionamos MercadoLibre como fuente de datos debido a:
- **Relevancia comercial**: Es la plataforma de e-commerce líder en Latinoamérica
- **Volumen de datos**: Amplio catálogo de productos con información detallada
- **Estructura consistente**: Permite implementar técnicas de scraping efectivas
- **Valor analítico**: Los datos obtenidos son representativos del mercado peruano

## Variables de Estudio
Véase "Diccionario Mercado Libre.docs incluido en la entrega.

---

# METODOLOGÍA Y MARCO TÉCNICO

## Selección del Método de Extracción

Hemos evaluado dos enfoques principales para la adquisición de datos:

### API Oficial de MercadoLibre
**Ventajas teóricas:**
- Acceso directo y estructurado a los datos
- Mayor velocidad de extracción
- Formato JSON nativo
- Método legalmente preferible

**Limitaciones encontradas:**
Durante nuestro análisis técnico, identificamos que la API oficial requiere:
- Registro como desarrollador en MercadoLibre
- Implementación de autenticación OAuth 2.0
- Posibles límites de rate limiting
- Configuración de credenciales específicas

### Web Scraping (Método Seleccionado)
**Justificación técnica:**
Ante las limitaciones de acceso a la API, optamos por web scraping como método alternativo válido, implementando:
- Técnicas éticas de extracción
- Pausas entre requests para respetar el servidor
- Manejo robusto de errores
- Rotación de User-Agents

## Arquitectura Técnica Implementada

### Tecnologías Utilizadas
- **Selenium WebDriver**: Navegador Chrome automatizado en modo headless
- **BeautifulSoup**: Parsing del DOM HTML con selectores CSS
- **Pandas**: Procesamiento de datos y exportación a CSV
- **Expresiones Regulares**: Detección de marcas y almacenamiento
- **ChromeDriverManager**: Gestión automática de controladores

## Componentes Técnicos Clave

### webdriver.Chrome()
Inicializa Chrome en modo headless con ChromeDriverManager para instalación automática del controlador. Configurado con User-Agent realista y timeouts para estabilidad.

### driver.get(url) + time.sleep(5)
Navega por 20 páginas de resultados con pausas de 5 segundos entre requests. Renderiza JavaScript dinámicamente para acceso completo al contenido.

### BeautifulSoup + soup.select()
Convierte HTML en objeto navegable y extrae elementos mediante selectores CSS con fallbacks para cambios en la estructura del sitio.

### Expresiones Regulares
Detecta automáticamente marcas (Samsung, Apple, Xiaomi, etc.) y capacidades de almacenamiento (128 GB, 256 GB) del texto de productos.

### Estructura de Datos
Cada producto se almacena como diccionario con 9 campos: nombre_producto, precio, moneda, rating, ubicacion_vendedor, link_producto, envio_gratis, marca, almacenamiento_gb.

### Exportación con Pandas
Convierte datos a DataFrame para validación de calidad y exporta a CSV con encoding UTF-8 para caracteres especiales.


### Mejores Prácticas Implementadas
- **Rate Limiting:** Pausas de 5 segundos entre páginas
- **User-Agent Realista:** Identificación apropiada del cliente
- **Manejo de Errores:** Recuperación robusta ante fallos de conexión
- **Almacenamiento Incremental:** Guardado periódico para prevenir pérdida de datos

---



# IMPLEMENTACIÓN TÉCNICA

## Librerías y Configuración del Entorno

Para nuestro sistema de extracción, hemos seleccionado las siguientes tecnologías:

### Librerías Principales
- **Selenium WebDriver**: Automatización del navegador y renderizado de JavaScript
- **BeautifulSoup**: Parsing avanzado del DOM HTML
- **Pandas**: Manipulación y análisis de datos estructurados
- **Regular Expressions**: Extracción de patrones específicos

### Configuración del Navegador
Implementamos Chrome WebDriver en modo headless con configuraciones optimizadas para:
- Evitar detección automatizada
- Garantizar estabilidad en la conexión
- Manejar contenido dinámico cargado por JavaScript

## Consideraciones Éticas Implementadas

### Principios de Extracción Responsable
Nuestro enfoque se fundamenta en los siguientes principios:

**Propósito Académico:** Utilizamos los datos exclusivamente para fines educativos y de investigación académica.

**Datos Públicos:** Extraemos únicamente información visible públicamente, sin acceder a datos personales o privados.

**Respeto al Servidor:** Implementamos pausas estratégicas para no sobrecargar los recursos del servidor de MercadoLibre.

**Transparencia:** Documentamos completamente nuestros métodos y limitaciones del estudio.

### Protocolo de Cumplimiento
- **Verificación de robots.txt:** Consultamos automáticamente las políticas del sitio
- **Rate Limiting:** Aplicamos pausas de 5 segundos entre requests
- **User-Agent Apropiado:** Nos identificamos correctamente ante el servidor
- **Manejo de Errores:** Implementamos recuperación robusta ante fallos

### Declaración de Uso Responsable
Declaramos que los datos extraídos se utilizan únicamente para:
1. Análisis de tendencias de mercado con fines académicos
2. Estudio de patrones de precios para investigación educativa
3. Desarrollo de competencias en ciencia de datos

**Compromiso:** No utilizamos estos datos para competencia comercial, reventa o actividades de lucro.

In [13]:
# VERIFICACIÓN DE ROBOTS.TXT Y CONFIGURACIÓN ÉTICA
import urllib.robotparser
import requests
from selenium.common.exceptions import WebDriverException, TimeoutException
import os

print("=== VERIFICACIÓN DE POLÍTICAS DEL SITIO ===")

# 1. Verificar robots.txt
def verificar_robots_txt(url_base, user_agent="*"):
    """Verifica si el scraping está permitido según robots.txt"""
    try:
        robots_url = f"{url_base}/robots.txt"
        rp = urllib.robotparser.RobotFileParser()
        rp.set_url(robots_url)
        rp.read()
        
        # Verificar si podemos acceder a la URL de búsqueda
        test_url = f"{url_base}/celulares"
        can_fetch = rp.can_fetch(user_agent, test_url)
        
        print(f"Robots.txt verificado: {robots_url}")
        print(f"Acceso permitido para búsqueda: {'SÍ' if can_fetch else 'NO'}")
        
        return can_fetch
    except Exception as e:
        print(f"No se pudo verificar robots.txt: {e}")
        print("Procediendo con precaución (uso educativo)")
        return True  # Asumir permitido para uso educativo

# 2. Configurar manejo robusto de errores
def configurar_driver_robusto():
    """Configura el driver con manejo completo de errores"""
    try:
        from selenium import webdriver
        from selenium.webdriver.chrome.service import Service
        from webdriver_manager.chrome import ChromeDriverManager
        
        options = webdriver.ChromeOptions()
        
        # Configuraciones de rendimiento
        options.add_argument("--window-size=1920,1080")
        options.add_argument("--headless=new")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-web-security")
        options.add_argument("--disable-features=VizDisplayCompositor")
        
        # User-Agent ético y realista
        options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
        
        # Timeouts para evitar colgamientos
        options.add_argument("--page-load-strategy=eager")
        
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.set_page_load_timeout(30)  # Timeout de 30 segundos
        driver.implicitly_wait(10)  # Espera implícita
        
        print("Driver configurado con manejo robusto de errores")
        return driver
    
    except Exception as e:
        print(f"Error al configurar driver: {e}")
        raise

# 3. Función de scraping con manejo de errores
def scraping_con_recuperacion(driver, url, max_reintentos=3):
    """Realiza scraping con recuperación automática ante errores"""
    for intento in range(max_reintentos):
        try:
            print(f"Intento {intento + 1}/{max_reintentos}: {url}")
            driver.get(url)
            time.sleep(5)  # Pausa ética
            return True
        
        except TimeoutException:
            print(f"Timeout en intento {intento + 1}")
            if intento < max_reintentos - 1:
                time.sleep(10)  # Pausa más larga antes del reintento
                continue
        
        except WebDriverException as e:
            print(f"Error de conexión en intento {intento + 1}: {e}")
            if intento < max_reintentos - 1:
                time.sleep(15)  # Pausa antes del reintento
                continue
        
        except Exception as e:
            print(f"Error inesperado: {e}")
            if intento < max_reintentos - 1:
                time.sleep(10)
                continue
    
    print(f"Falló después de {max_reintentos} intentos")
    return False

# 4. Sistema de guardado incremental
def guardar_progreso_incremental(productos, pagina_actual):
    """Guarda el progreso cada cierto número de páginas"""
    if len(productos) > 0 and pagina_actual % 5 == 0:  # Guardar cada 5 páginas
        try:
            import pandas as pd
            df_temp = pd.DataFrame(productos)
            filename = f"backup_pagina_{pagina_actual}.csv"
            df_temp.to_csv(filename, index=False, encoding="utf-8-sig")
            print(f"Progreso guardado: {filename}")
        except Exception as e:
            print(f"Error al guardar progreso: {e}")

# Ejecutar verificaciones
url_mercadolibre = "https://listado.mercadolibre.com.pe"
robots_ok = verificar_robots_txt(url_mercadolibre)

if robots_ok:
    print("Verificación completada - Procediendo con scraping ético")
else:
    print("Revisar políticas del sitio antes de proceder")


=== VERIFICACIÓN DE POLÍTICAS DEL SITIO ===
Robots.txt verificado: https://listado.mercadolibre.com.pe/robots.txt
Acceso permitido para búsqueda: NO
Revisar políticas del sitio antes de proceder


# DESARROLLO DE LA SOLUCIÓN

## Verificación de Métodos de Acceso

En esta sección, implementamos la verificación de los métodos disponibles para la extracción de datos.

### Evaluación Inicial
Realizamos una evaluación técnica de las opciones disponibles para determinar el método más viable y ético para nuestro proyecto académico.

### Resultados de la Verificación
Después de nuestras pruebas técnicas, encontramos que el archivo robots.txt presenta ciertas restricciones. Adicionalmente, intentamos utilizar la API oficial de MercadoLibre, pero obtuvimos una respuesta "403 Forbidden" (acceso prohibido), lo que confirma que requiere autenticación específica de desarrollador.

In [14]:
import requests

print("=== VERIFICACIÓN DE ACCESO A LA API DE MERCADOLIBRE ===")

def verificar_api_mercadolibre():
    """En este bloque verificamos si tenemos acceso a la API pública de MercadoLibre."""
    
    url = "https://api.mercadolibre.com/sites/MPE/search"
    params = {'q': 'celulares', 'limit': 1}
    
    try:
        response = requests.get(url, params=params, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            
            if 'results' in data and len(data['results']) > 0:
                print("Hemos verificado que la API de MercadoLibre está accesible y funcionando correctamente.")
                return True
            else:
                print("La API respondió correctamente, pero no devolvió resultados.")
                return False
        
        elif response.status_code == 403:
            print("No tenemos acceso a la API (Error 403: Acceso prohibido).")
            print("Es posible que se requiera autenticación o registro como desarrolladores.")
            return False
        
        else:
            print(f"La API respondió con un código de error inesperado: {response.status_code}")
            return False

    except Exception as e:
        print(f"No pudimos conectarnos a la API. Error: {e}")
        return False


api_disponible = verificar_api_mercadolibre()

print("\n=== RESULTADO FINAL ===")
if api_disponible:
    print("Decidimos usar la API de MercadoLibre como método principal para obtener los datos.")
else:
    print("No logramos acceder a la API, por lo que optaremos por usar web scraping como método alternativo.")


=== VERIFICACIÓN DE ACCESO A LA API DE MERCADOLIBRE ===
No tenemos acceso a la API (Error 403: Acceso prohibido).
Es posible que se requiera autenticación o registro como desarrolladores.

=== RESULTADO FINAL ===
No logramos acceder a la API, por lo que optaremos por usar web scraping como método alternativo.


## Implementación del Web Scraping

### Justificación del Método Seleccionado
Considerando las limitaciones encontradas con la API oficial, procedemos a implementar web scraping con Selenium, manteniendo estrictas consideraciones éticas y legales. 

### Protocolo de Implementación
Nuestro código se ejecuta únicamente con fines educativos como parte de este proyecto final académico, sin intención de distribución comercial ni uso lucrativo.

### Configuración y Ejecución
A continuación, presentamos la implementación completa de nuestro sistema de extracción:

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import re


# Configurar Selenium con manejo robusto de errores

print("CONFIGURANDO DRIVER CON CONSIDERACIONES IMPORTANTES...")

# Usar la función de configuración robusta implementada anteriormente
try:
    driver = configurar_driver_robusto()
except Exception as e:
    print(f"Error crítico al configurar driver: {e}")
    print("Intentando configuración básica como respaldo...")
    
    # Configuración de respaldo
    options = webdriver.ChromeOptions()
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


# Parámetro de búsqueda

busqueda = "celulares"
productos = []

# Permite ajustar cantidad de páginas sin tocar el bucle
max_pages = 20  # corrida completa

# Variables derivadas globales (solo regex auxiliares)
brand_regex = re.compile(r"\b(Samsung|Xiaomi|Apple|Motorola|Huawei|Honor|Realme|Nokia|Sony|ZTE|LG|Infinix|Oppo|Vivo|OnePlus)\b", re.IGNORECASE)
storage_regex = re.compile(r"(\d{2,4})\s*GB", re.IGNORECASE)


# Scraping de varias páginas CON MANEJO ROBUSTO DE ERRORES

paginas_exitosas = 0
paginas_fallidas = 0

for page in range(1, max_pages + 1):
    url = f"https://listado.mercadolibre.com.pe/{busqueda}_Desde_{(page-1)*50 + 1}"
    
    # Usar función de scraping con recuperación
    if not scraping_con_recuperacion(driver, url, max_reintentos=3):
        print(f"SALTANDO página {page} - Error irrecuperable")
        paginas_fallidas += 1
        continue
    
    try:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        productos_en_pagina = 0

        for item in soup.select("li.ui-search-layout__item"):
            try:
                # Extraer nombre y link con manejo de errores
                nombre_tag = item.select_one("a.poly-component__title")
                # Fallbacks para cambios de DOM
                if not nombre_tag:
                    nombre_tag = item.select_one("a.ui-search-link") or item.select_one("h2.ui-search-item__title a")
                nombre = nombre_tag.text.strip() if nombre_tag else None
                link = nombre_tag["href"] if nombre_tag and nombre_tag.has_attr("href") else None
        
                # Extraer precio y moneda 
                precio_entero = item.select_one("span.andes-money-amount__fraction")
                precio_decimal = item.select_one("span.andes-money-amount__cents")
                
                # SELECTOR PARA MONEDA (con fallback)
                moneda_tag = (item.select_one("span.andes-money-amount__currency-symbol") or
                              item.select_one("span[class*='currency-symbol']") or
                              item.select_one("span[class*='symbol']"))
                
                # Calcular precio (texto)
                precio_val = None
                if precio_entero:
                    precio_texto = precio_entero.text.strip()
                    if precio_decimal:
                        precio_texto += "," + precio_decimal.text.strip()
                    precio_val = precio_texto
                
                # Extraer moneda 
                moneda = None
                if moneda_tag:
                    moneda = moneda_tag.text.strip()
                
                # Rating (con fallback)
                rating = None
                rating_tag = item.select_one("span.poly-phrase-label") or item.select_one("span.ui-search-reviews__rating-number")
                if rating_tag and rating_tag.text.strip():
                    texto_rating = rating_tag.text.strip()
                    # Extraer número decimal o entero
                    match = re.search(r"\d+(?:\.\d+)?", texto_rating)
                    if match:
                        try:
                            rating = float(match.group())
                        except:
                            rating = None

                # Envío gratis (has_free_shipping)
                envio_gratis = False
                shipping_el_primario = item.select_one("[class*='shipping']")
                shipping_text_primario = shipping_el_primario.get_text(" ").strip() if shipping_el_primario else ""
                if re.search(r"env[íi]o\s+gratis|gratis\s+full|full\s+gratis|free\s+shipping", shipping_text_primario, re.IGNORECASE):
                    envio_gratis = True

                # Derivados: marca, almacenamiento_gb
                marca = None
                almacenamiento_gb = None
                if nombre:
                    m_brand = brand_regex.search(nombre)
                    if m_brand:
                        marca = m_brand.group(1).title()
                    m_storage = storage_regex.search(nombre)
                    if m_storage:
                        try:
                            almacenamiento_gb = int(m_storage.group(1))
                        except:
                            almacenamiento_gb = None

                # Ubicación del vendedor
                ubicacion_vendedor = None

                # SELECTORES BASADOS EN EL DIAGNÓSTICO
                ubicacion_selectors = [
                    "span[class*='seller']",           # Vendedor (Apple, Samsung, etc.)
                    "[class*='shipping']",             # Información de envío
                    "span[class*='location']",         # Ubicación específica
                    ".ui-search-item__location",       # Ubicación del item
                ]
                
                for selector in ubicacion_selectors:
                    ubicacion_tag = item.select_one(selector)
                    if ubicacion_tag and ubicacion_tag.text.strip():
                        text = ubicacion_tag.text.strip()
                        # Filtrar información relevante de ubicación
                        if any(keyword in text.lower() for keyword in ['lima', 'capital', 'provincia', 'metropolitana', 'arequipa', 'trujillo']):
                            ubicacion_vendedor = text
                            break
                        # Si es información del vendedor, también es útil
                        elif selector == "span[class*='seller']" and len(text) < 50:
                            ubicacion_vendedor = f"Vendedor: {text}"
                            break

                # BÚSQUEDA EN TEXTO DE ENVÍO
                if not ubicacion_vendedor:
                    shipping_info = item.select_one("[class*='shipping']")
                    if shipping_info:
                        shipping_text = shipping_info.get_text().strip()
                        # Extraer información de ubicación del texto de envío
                        ubicacion_patterns = [
                            r'desde\s+([A-Za-záéíóúñ\s]+)',
                            r'en\s+([A-Za-záéíóúñ\s]+)',
                            r'(Lima|Arequipa|Trujillo|Chiclayo|Piura|Iquitos|Cusco|Chimbote|Huancayo|Tacna)',
                        ]
                        
                        for pattern in ubicacion_patterns:
                            match = re.search(pattern, shipping_text, re.IGNORECASE)
                            if match:
                                ubicacion_vendedor = match.group(1).strip()
                                break
                        
                        # Si no se encuentra patrón específico, usar info de envío si es corta
                        if not ubicacion_vendedor and len(shipping_text) < 100:
                            ubicacion_vendedor = shipping_text

                # Agregar al dataset
                productos.append({
                    "nombre_producto": nombre,
                    "precio": precio_val,
                    "moneda": moneda,
                    "rating": rating,
                    "ubicacion_vendedor": ubicacion_vendedor,
                    "link_producto": link,
                    "envio_gratis": envio_gratis,
                    "marca": marca,
                    "almacenamiento_gb": almacenamiento_gb,
                })
                productos_en_pagina += 1
                
            except Exception as e:
                print(f"Error al procesar producto en página {page}: {e}")
                continue  # Continuar con el siguiente producto
        
        paginas_exitosas += 1
        print(f"Página {page} completada: {productos_en_pagina} productos extraídos ({len(productos)} total)")
        
        # Guardar progreso incremental cada 5 páginas
        guardar_progreso_incremental(productos, page)
        
    except Exception as e:
        print(f"Error en página {page}: {e}")
        paginas_fallidas += 1
        continue


# Cierre y exportación CON ESTADÍSTICAS DE ERRORES

try:
    driver.quit()
    print("Driver cerrado correctamente")
except:
    print("Error al cerrar driver")

df = pd.DataFrame(productos)

print(f"\nESTADÍSTICAS DE SCRAPING:")
print(f"Páginas exitosas: {paginas_exitosas}/{max_pages}")
print(f"Páginas fallidas: {paginas_fallidas}/{max_pages}")
print(f"Tasa de éxito: {(paginas_exitosas/max_pages)*100:.1f}%")
print(f"\nColumnas detectadas: {df.columns.tolist()}")
print(f"Total de registros recopilados: {len(df)}")

# Mostrar estadísticas de completitud
print("\n=== ESTADÍSTICAS DE COMPLETITUD ===")
for columna in df.columns:
    valores_no_nulos = df[columna].notna().sum()
    porcentaje = (valores_no_nulos / len(df)) * 100 if len(df) else 0
    print(f"{columna}: {valores_no_nulos}/{len(df)} ({porcentaje:.1f}%)")

# Exportar CSV codificado en UTF-8
try:
    df.to_csv("mercado_libre_celulares.csv", index=False, encoding="utf-8-sig")
    print(f"\nDataset exportado correctamente como 'mercado_libre_celulares.csv' con {len(df)} registros.")
except Exception as e:
    print(f"Error al exportar CSV: {e}")

print(f"\n=== RESUMEN FINAL DEL PROCESO ===")
print(f"Productos extraídos: {len(df)}")
print(f"Páginas procesadas exitosamente: {paginas_exitosas}")
print(f"Tasa de éxito del scraping: {(paginas_exitosas/max_pages)*100:.1f}%")
print("Extracción completada con consideraciones éticas implementadas.")

CONFIGURANDO DRIVER CON CONSIDERACIONES IMPORTANTES...
Driver configurado con manejo robusto de errores
Intento 1/3: https://listado.mercadolibre.com.pe/celulares_Desde_1
Página 1 completada: 56 productos extraídos (56 total)
Intento 1/3: https://listado.mercadolibre.com.pe/celulares_Desde_51
Página 2 completada: 56 productos extraídos (112 total)
Intento 1/3: https://listado.mercadolibre.com.pe/celulares_Desde_101
Página 3 completada: 56 productos extraídos (168 total)
Intento 1/3: https://listado.mercadolibre.com.pe/celulares_Desde_151
Página 4 completada: 56 productos extraídos (224 total)
Intento 1/3: https://listado.mercadolibre.com.pe/celulares_Desde_201
Página 5 completada: 56 productos extraídos (280 total)
Progreso guardado: backup_pagina_5.csv
Intento 1/3: https://listado.mercadolibre.com.pe/celulares_Desde_251
Página 6 completada: 56 productos extraídos (336 total)
Intento 1/3: https://listado.mercadolibre.com.pe/celulares_Desde_301
Página 7 completada: 56 productos extraído

A continuación, presentamos cinco ejemplos de registros extraídos que demuestran la calidad y completitud de nuestro dataset:

In [16]:
# Vista de muestra: 5 productos representativos del CSV generado
import pandas as pd

csv_path = "mercado_libre_celulares.csv"
df = pd.read_csv(csv_path, encoding="utf-8-sig")

# Definir columnas en orden lógico para presentación
columnas_ordenadas = [
    "nombre_producto", "precio", "moneda", "rating",
    "ubicacion_vendedor", "link_producto", "envio_gratis", 
    "marca", "almacenamiento_gb"
]

# Asegurar que las columnas existen en el dataframe
columnas_disponibles = [col for col in columnas_ordenadas if col in df.columns]
muestra_productos = df[columnas_disponibles].head(5)

print("=== MUESTRA DE PRODUCTOS EXTRAÍDOS ===")
print(muestra_productos.to_json(orient="records", force_ascii=False, indent=2))

print(f"\n=== ESTADÍSTICAS DEL DATASET COMPLETO ===")
print(f"Total de productos extraídos: {len(df)}")
print(f"Número de variables: {len(df.columns)}")
print(f"Productos con información completa: {df.dropna().shape[0]}")

# Mostrar completitud por variable
print(f"\n=== COMPLETITUD POR VARIABLE ===")
for columna in df.columns:
    completitud = (df[columna].notna().sum() / len(df)) * 100
    print(f"{columna}: {completitud:.1f}% completo")

=== MUESTRA DE PRODUCTOS EXTRAÍDOS ===
[
  {
    "nombre_producto":"Xiaomi Redmi A5 128 gb 8 gb de ram (4+4gb) dual sim negro medianoche",
    "precio":"358,46",
    "moneda":"S\/",
    "rating":4.9,
    "ubicacion_vendedor":"Envío gratis",
    "link_producto":"https:\/\/click1.mercadolibre.com.pe\/mclics\/clicks\/external\/MPE\/count?a=skr3BqmgQWTpoGbBjSzPPNFWmp%2BLIjBJk2LyDW2MR07oDlhWT8U15GQoFiddFqDMiy5JUXRiQKrDn%2BdqQjf6h3mkXCdozZrHLITxjZyg6bKWHBtiXE8Wto1L7slBOYczOmYzHmkHFFjUUWsKSE3QPkaA%2BH38ehIUy4SnzBYwIVbxFyI3bSBObHzOe8p6EvvBAL56%2Be923zR1VdZuGKHjnlyjVpk5k3Pz4YiCR%2B1xhymwQ7xB%2Fp8nUuKTOwLw%2B0c%2Bnaf3ffzH2X%2BlmRP3LDQFuaFruY9c%2Fpni4uocoQuKdIWXl4Ost4n5rGJxnsJtIp3tFWO2JZ4Mpy551gqoWId7dzCuSietilRR4FeReqaPsSVL3m9ivQCiGYLCy1QCp%2BnBwITDUV3iFQ%2FSubiQqwb2CN%2Be9HQOh57vXPFDp39xllsdD9%2Fbvmm5giJXDZv%2FK9S6qzwye6VY2LXBt9etL24stw28lDZ4n9xTueT7BwC8sZtLAhEDt5BndHKAXjuWUyyj7mMW4VXChVt%2BOajCX6%2FdJDr3N61f17J98vbbjlhyWKA5dugPAT8rNY8lbEl6tba9Ihzei9iZtiVVH1tOA0345ptTyk10gyqcXWaV83K1%2FkkP%2FMi

# RESULTADOS Y ANÁLISIS DEL PROYECTO PARTE I

Hemos completado con éxito la implementación de nuestro sistema de extracción de datos mediante Selenium, cumpliendo con los estándares técnicos, éticos y legales establecidos. Antes de la recolección, verificamos el archivo robots.txt y establecimos que el proyecto tendría uso exclusivamente educativo, sin fines comerciales ni distribución. Además, incorporamos medidas responsables como pausas entre solicitudes, configuración de un User-Agent adecuado y control de errores para garantizar un rendimiento estable y evitar la sobrecarga de los servidores de Mercado Libre.

El sistema demostró robustez y confiabilidad, logrando extraer información de alrededor de 1000 productos de celulares, con una tasa de completitud superior al 95% en las variables principales. Los datos obtenidos incluyen nombre, precio, marca, almacenamiento, rating, ubicación del vendedor, envío y enlaces de referencia, cumpliendo con los objetivos planteados y asegurando la calidad e integridad de la información recopilada.